In [19]:
# ToDo 

# 汎用ライブラリのimport
import sys
import os
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [20]:
# 自作ライブラリのimport
if os.environ['BR_HOME']+"/boatrace" not in sys.path:
    sys.path.append(os.environ['BR_HOME']+"/boatrace")
#print(sys.path)

from setup.myUtil import dbHandler

In [21]:
# 分析期間の指定は一旦ここでまとめてみる。
analyzeStartDate="20180101"
analyzeEndDate="20180131"

In [22]:
dbh=dbHandler.getDBHandle()
#dbHandler.closeDBHandle(dbh)

In [23]:
# 日付と登録番号を与えて、該当選手の直近30回のスタート時刻の、各レース平均に対する偏差平均を出す。フライング除く。
# 現在は、各レース平均に対する偏差のリストを返している。
raceDate = analyzeEndDate
toban="4337"
with dbh.cursor() as cursor:
    sel_sql = "select v.raceId,(s.startTime-v.starttime_avg) as startTime_dev,s.startTime,v.starttime_avg \
               from raceresult s,racemanagement m, \
                    (select raceId,avg(startTime) as starttime_avg from raceresult \
                     where startTime between 0 and 9.9 \
                     group by raceId \
                     having count(*) = 6 ) v \
               where s.raceId = v.raceId \
                 and s.raceId = m.raceId \
                 and s.toban = '%s' \
                 and m.raceDate < '%s' \
                 order by raceId desc \
                 limit 30 "\
               % (toban,raceDate)
    cursor.execute(sel_sql)
    startInfoList=cursor.fetchall()
print(len(startInfoList))
#print(startInfoList)

23


In [24]:
# 他の人よりどれだけスタートが早いと、進入コース別に、結果（1着、2着、3着、それ以下）にどれだけ影響があるのか。


In [25]:
# 進入コースとスタート偏差にはどれだけ関係があるのか。アウトになるほどばらつきが大きくなる？
with dbh.cursor() as cursor:
    sel_sql = "select v.raceId,(s.startTime-v.starttime_avg) as startTime_dev,s.startLane,s.goalRank,s.lane \
               from raceresult s, racemanagement m, \
                    (select raceId,avg(startTime) as starttime_avg from raceresult \
                     where startTime between 0 and 9.9 \
                     group by raceId \
                     having count(*) = 6 ) v \
               where s.raceId = v.raceId \
                 and s.raceId = m.raceId \
                 and m.raceDate between '%s' and '%s' " \
              % (analyzeStartDate,analyzeEndDate)
    cursor.execute(sel_sql)
    startInfoList=cursor.fetchall()
print(len(startInfoList))
#print(startInfoList)


29124


In [26]:
df = pd.io.json.json_normalize(startInfoList)
df.head()

,goalRank,lane,raceId,startLane,startTime_dev
0,1,2,20180101-06-01,2,-0.066667
1,2,1,20180101-06-01,1,-0.056667
2,3,3,20180101-06-01,3,-0.056667
3,4,4,20180101-06-01,4,0.023333
4,5,5,20180101-06-01,5,0.173333


In [28]:
# まずは素朴に順位別にstartTime_devを見てみる。
# 着順別の傾向。明らかに、ゴールした順位とスタートタイムは影響する。0.01秒でも十分な違いと見て良さそう。
groupeddf=df.groupby('goalRank')
groupeddf.describe()

lane                                              startLane  \
           count      mean       std  min  25%  50%  75%  max     count   
goalRank                                                                  
1         4854.0  2.121961  1.466839  1.0  1.0  1.0  3.0  6.0    4854.0   
2         4854.0  3.056242  1.521848  1.0  2.0  3.0  4.0  6.0    4854.0   
3         4854.0  3.618459  1.560735  1.0  2.0  4.0  5.0  6.0    4854.0   
4         4854.0  3.894726  1.578185  1.0  3.0  4.0  5.0  6.0    4854.0   
5         4854.0  4.064689  1.572330  1.0  3.0  4.0  5.0  6.0    4854.0   
6         4854.0  4.243923  1.580663  1.0  3.0  5.0  6.0  6.0    4854.0   

                    ...           startTime_dev                                \
              mean  ...  75%  max         count      mean       std       min   
goalRank            ...                                                         
1         2.080758  ...  3.0  6.0        4854.0 -0.029108  0.041357 -0.195000   
2         3.014421  ...  4.0  6.0        4854.0 -0.012872  0.041296 -0.190000   
3         3.579110  ...  5.0  6.0        4854.0 -0.000857  0.044010 -0.186667   
4         3.885867  ...  5.0  6.0        4854.0  0.006011  0.046596 -0.148333   
5         4.079316  ...  5.0  6.0        4854.0  0.014388  0.050459 -0.191667   
6         4.360527  ...  6.0  6.0        4854.0  0.022439  0.056625 -0.166667   

                                                  
               25%       50%       75%       max  
goalRank                                          
1        -0.055000 -0.028333 -0.002084  0.143333  
2        -0.038333 -0.013333  0.011667  0.218333  
3        -0.028333 -0.003333  0.023333  0.505000  
4        -0.023333  0.003333  0.031667  0.295000  
5        -0.018333  0.010000  0.040000  0.580000  
6        -0.013333  0.015000  0.050000  0.536667  

[6 rows x 24 columns]

In [29]:
# 上記をグラフ化。グラフにするとあまり見えない。
#df[['goalRank','startTime_dev']].plot(kind='scatter',x='goalRank',y='startTime_dev')

In [30]:
# まずは素朴に順位別にstartTime_devを見てみる。
# 進入コース別の傾向。平均で、1レーンからスタートが早く、順に6レーンは遅い傾向がある。インの方が合わせやすいということか。
# しかし標準偏差ははっきりしない。外側だとぶれやすいわけではない。
groupeddf=df.groupby('startLane')
groupeddf.describe()

goalRank                                                 lane  \
             count      mean       std  min  25%  50%  75%  max   count   
startLane                                                                 
1           4854.0  2.142151  1.566568  1.0  1.0  1.0  3.0  6.0  4854.0   
2           4854.0  3.233828  1.585895  1.0  2.0  3.0  5.0  6.0  4854.0   
3           4854.0  3.428513  1.582454  1.0  2.0  3.0  5.0  6.0  4854.0   
4           4854.0  3.545529  1.591203  1.0  2.0  4.0  5.0  6.0  4854.0   
5           4854.0  4.027194  1.488496  1.0  3.0  4.0  5.0  6.0  4854.0   
6           4854.0  4.622785  1.349674  1.0  4.0  5.0  6.0  6.0  4854.0   

                     ...           startTime_dev                      \
               mean  ...  75%  max         count      mean       std   
startLane            ...                                               
1          1.036259  ...  1.0  6.0        4854.0 -0.013391  0.052951   
2          2.136794  ...  2.0  6.0        4854.0  0.000138  0.047509   
3          3.066131  ...  3.0  6.0        4854.0 -0.002351  0.046815   
4          4.050680  ...  4.0  6.0        4854.0 -0.001366  0.044485   
5          4.980016  ...  5.0  6.0        4854.0  0.001363  0.045773   
6          5.730119  ...  6.0  6.0        4854.0  0.015607  0.057255   

                                                             
                min       25%       50%       75%       max  
startLane                                                    
1         -0.183333 -0.046667 -0.016667  0.015000  0.536667  
2         -0.176667 -0.030000 -0.003333  0.026667  0.501667  
3         -0.186667 -0.030000 -0.006667  0.020000  0.580000  
4         -0.195000 -0.030000 -0.003333  0.023333  0.410000  
5         -0.190000 -0.026667  0.000000  0.028333  0.391667  
6         -0.191667 -0.021667  0.011667  0.050000  0.390000  

[6 rows x 24 columns]

In [13]:
# まずは素朴に順位別にstartTime_devを見てみる。
# 枠別の傾向。進入コース別と同じ傾向。枠と進入コースがほぼ同等？なので、当然といえば当然。
# ToDo 枠と進入コースの関係を表示する
groupeddf=df.groupby('lane')
groupeddf.describe()

goalRank                                              startLane  \
        count      mean       std  min  25%  50%  75%  max     count   
lane                                                                   
1     13145.0  2.129099  1.560845  1.0  1.0  1.0  3.0  6.0   13145.0   
2     13145.0  3.255306  1.609400  1.0  2.0  3.0  5.0  6.0   13145.0   
3     13145.0  3.447471  1.586231  1.0  2.0  3.0  5.0  6.0   13145.0   
4     13145.0  3.659414  1.583836  1.0  2.0  4.0  5.0  6.0   13145.0   
5     13145.0  4.114492  1.503462  1.0  3.0  4.0  5.0  6.0   13145.0   
6     13145.0  4.394218  1.430026  1.0  3.0  5.0  6.0  6.0   13145.0   

                ...           startTime_dev                                \
          mean  ...  75%  max         count      mean       std       min   
lane            ...                                                         
1     1.020844  ...  1.0  6.0       13145.0 -0.012689  0.052768 -0.200000   
2     2.092507  ...  2.0  6.0       13145.0  0.000209  0.047761 -0.200000   
3     3.095474  ...  3.0  6.0       13145.0 -0.002958  0.045706 -0.186667   
4     4.126664  ...  4.0  6.0       13145.0 -0.001641  0.045298 -0.216667   
5     4.985546  ...  5.0  6.0       13145.0  0.003540  0.048676 -0.205000   
6     5.678965  ...  6.0  6.0       13145.0  0.013538  0.056127 -0.195000   

                                              
           25%       50%       75%       max  
lane                                          
1    -0.046667 -0.016667  0.016667  0.536667  
2    -0.030000 -0.003333  0.026667  0.501667  
3    -0.030000 -0.006667  0.020000  0.588333  
4    -0.030000 -0.003333  0.023333  0.488333  
5    -0.026667  0.001667  0.030000  0.505000  
6    -0.023333  0.010000  0.045000  0.700000  

[6 rows x 24 columns]

In [14]:
# 枠と進入コースがどれだけ一致するかをチェックする。


In [15]:
# 過去のstartTime_devから未来は予測できるのか？自己相関があるか、どのくらいの期間影響するのか、を検証。
# 30レース以上に出場している選手に限定する ←データが集まってないのでいったん、20レースにする。
with dbh.cursor() as cursor:
    sel_sql = "select (s.startTime-v.starttime_avg) as startTime_dev,s.toban \
               from raceresult s,racemanagement m, \
                    (select toban from raceresult \
                     where startTime between 0 and 9.9 \
                     group by toban having count(*) > 20) t,\
                    (select raceId,avg(startTime) as starttime_avg from raceresult \
                     where startTime between 0 and 9.9 \
                     group by raceId \
                     having count(*) = 6 ) v \
               where s.raceId = v.raceId \
                 and s.toban  = t.toban \
                 and s.raceId = m.raceId \
                 and m.raceDate between '%s' and '%s' \
               order by s.toban,s.raceId desc" \
              % (analyzeStartDate,analyzeEndDate)
    cursor.execute(sel_sql)
    startInfoList=cursor.fetchall()
print(len(startInfoList))
#print(startInfoList)
df = pd.io.json.json_normalize(startInfoList)
df.head()

78188


,startTime_dev,toban
0,-0.056667,2014
1,-0.050000,2014
2,-0.105000,2014
3,-0.038333,2014
4,0.068333,2014


In [16]:
groupeddf=df.groupby('toban')
groupeddf.head()

,startTime_dev,toban
0,-0.056667,2014
1,-0.050000,2014
2,-0.105000,2014
3,-0.038333,2014
4,0.068333,2014
52,0.018333,2538
53,0.036667,2538
54,0.055000,2538
55,0.008333,2538
56,-0.128333,2538


In [17]:
df[df['toban']=='4873'].head()
# todo 次に、自己相関を見てみる。

,startTime_dev,toban
72573,-0.066667,4873
72574,0.041667,4873
72575,0.070000,4873
72576,0.015000,4873
72577,0.018333,4873


In [18]:
# メモリ使用チェック
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_") and sys.getsizeof(eval(var_name)) > 1000: #ここだけアレンジ
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                       df|   5395076|
|            startInfoList|    651352|
